### Plan
#### 1. Data Collection
#### 2. Preprocessed data
#### 3. Feature engineering
#### 4. Modeling
#### 5. Evaluation

### 1. Data collection

### Sofascore

![SofaScore](./images/sofascore.png)


### Livescore

![LiveScore](./images/livescore.png)

### Whoscored

![WhoScored](./images/whoscored.png)


Scrape dc càng nhiều nguồn thì càng tốt

### 2. Preprocess data

Chuyển data về dạng bảng 

Vd

In [1]:
import pandas as pd
import random

# data = {
#     'Team A': ['Liverpool', 'Mancity', 'Mancity'],
#     'Team B': ['Arsenal', 'Real Madrid', 'Barcelona'],
#     'Goals A': [1, 2, 3],
#     'Goals B': [0, 1, 2],
#     'Shots A': [10, 15, 20],
#     'Shots B': [5, 10, 15],
#     'Shots on Target A': [5, 10, 15],
#     'Shots on Target B': [3, 6, 9],
#     'Possession % A': [55, 60, 65],
#     'Possession % B': [45, 40, 35],
#     'Passes A': [400, 500, 600],
#     'Passes B': [300, 200, 100],
#     'Fouls A': [10, 20, 30],
#     'Fouls B': [5, 10, 15],
#     'Yellow Cards A': [1, 2, 3],
#     'Yellow Cards B': [0, 1, 2],
#     'Red Cards A': [0, 0, 1],
#     'Red Cards B': [0, 0, 0],
# }
teams = ['Liverpool', 'Mancity', 'Arsenal', 'Chelsea', 'Tottenham', 'Real Madrid', 'Barcelona', 'Atletico Madrid', 'Juventus', 'AC Milan']
data = {
    'Time': pd.date_range(start='1/1/2020', periods=100, freq='D'),
    'Team A': [],
    'Team B': [],
    'Goals A': [random.randint(0, 5) for _ in range(100)],
    'Goals B': [random.randint(0, 5) for _ in range(100)],
    'Shots A': [random.randint(5, 25) for _ in range(100)],
    'Shots B': [random.randint(5, 25) for _ in range(100)],
    'Shots on Target A': [random.randint(1, 10) for _ in range(100)],
    'Shots on Target B': [random.randint(1, 10) for _ in range(100)],
    'Possession % A': [random.randint(40, 60) for _ in range(100)],
    'Possession % B': [],
    'Passes A': [random.randint(200, 800) for _ in range(100)],
    'Passes B': [random.randint(200, 800) for _ in range(100)],
    'Fouls A': [random.randint(5, 30) for _ in range(100)],
    'Fouls B': [random.randint(5, 30) for _ in range(100)],
    'Yellow Cards A': [random.randint(0, 5) for _ in range(100)],
    'Yellow Cards B': [random.randint(0, 5) for _ in range(100)],
    'Red Cards A': [random.randint(0, 2) for _ in range(100)],
    'Red Cards B': [random.randint(0, 2) for _ in range(100)],
}

# Ensure Team A and Team B are different
for _ in range(100):
    team_a, team_b = random.sample(teams, 2)
    data['Team A'].append(team_a)
    data['Team B'].append(team_b)
    data['Possession % B'].append(100 - data['Possession % A'][_])

df = pd.DataFrame(data)
df

,Time,Team A,Team B,Goals A,Goals B,Shots A,Shots B,Shots on Target A,Shots on Target B,Possession % A,Possession % B,Passes A,Passes B,Fouls A,Fouls B,Yellow Cards A,Yellow Cards B,Red Cards A,Red Cards B
0,2020-01-01,Atletico Madrid,Tottenham,5,1,15,13,2,10,45,55,744,602,23,25,5,4,2,0
1,2020-01-02,AC Milan,Liverpool,2,2,13,21,2,4,56,44,461,462,9,18,2,2,0,1
2,2020-01-03,Mancity,Barcelona,3,4,25,17,2,9,50,50,250,527,22,6,2,1,0,2
3,2020-01-04,Tottenham,Atletico Madrid,4,2,6,12,4,2,55,45,469,201,6,23,2,1,1,2
4,2020-01-05,Liverpool,AC Milan,3,3,14,19,8,2,43,57,373,550,15,29,2,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-04-05,Liverpool,Barcelona,5,1,22,15,3,3,47,53,712,782,6,9,4,3,2,2
96,2020-04-06,Chelsea,Atletico Madrid,5,4,20,19,9,9,53,47,245,393,29,17,4,2,1,1
97,2020-04-07,AC Milan,Tottenham,4,2,9,7,5,9,55,45,464,379,13,22,2,3,2,2
98,2020-04-08,Mancity,Liverpool,5,0,12,12,5,8,56,44,258,369,24,10,1,0,2,2


### 2. Feature engineering

Tính các chỉ số biểu thị phong độ của 2 đội như số bàn thắng trung bình trong 5 trận gần nhất, mean passes, mean possession, mean shots, ...

Lịch sử đối đầu: hiệu số thắng thua,...

In [2]:
# Calculate mean goals per match for the last 5 matches of each team
df = df.sort_values(by='Time')
df['Goals A Mean'] = df['Goals A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Goals B Mean'] = df['Goals B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

# Calculate mean possession % per match for the last 5 matches of each team
df['Possession % A Mean'] = df['Possession % A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Possession % B Mean'] = df['Possession % B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

# Calculate mean shots per match for the last 5 matches of each team
df['Shots A Mean'] = df['Shots A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Shots B Mean'] = df['Shots B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

# Calculate mean shots on target per match for the last 5 matches of each team
df['Shots on Target A Mean'] = df['Shots on Target A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Shots on Target B Mean'] = df['Shots on Target B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

# Calculate mean passes per match for the last 5 matches of each team
df['Passes A Mean'] = df['Passes A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Passes B Mean'] = df['Passes B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

# Calculate mean fouls per match for the last 5 matches of each team
df['Fouls A Mean'] = df['Fouls A'].groupby(df['Team A']).rolling(window=5).mean().reset_index(drop=True)
df['Fouls B Mean'] = df['Fouls B'].groupby(df['Team B']).rolling(window=5).mean().reset_index(drop=True)

df


,Time,Team A,Team B,Goals A,Goals B,Shots A,Shots B,Shots on Target A,Shots on Target B,Possession % A,...,Possession % A Mean,Possession % B Mean,Shots A Mean,Shots B Mean,Shots on Target A Mean,Shots on Target B Mean,Passes A Mean,Passes B Mean,Fouls A Mean,Fouls B Mean
0,2020-01-01,Atletico Madrid,Tottenham,5,1,15,13,2,10,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,AC Milan,Liverpool,2,2,13,21,2,4,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,Mancity,Barcelona,3,4,25,17,2,9,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,Tottenham,Atletico Madrid,4,2,6,12,4,2,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,Liverpool,AC Milan,3,3,14,19,8,2,43,...,51.6,51.4,19.0,16.8,3.4,6.6,525.2,501.2,16.2,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2020-04-05,Liverpool,Barcelona,5,1,22,15,3,3,47,...,49.6,48.4,16.6,12.4,5.6,5.8,606.6,541.4,15.8,20.0
96,2020-04-06,Chelsea,Atletico Madrid,5,4,20,19,9,9,53,...,47.2,45.8,17.6,13.8,5.0,5.6,645.2,483.6,18.6,17.2
97,2020-04-07,AC Milan,Tottenham,4,2,9,7,5,9,55,...,46.6,49.0,17.4,13.4,6.8,4.4,600.4,553.8,16.8,18.2
98,2020-04-08,Mancity,Liverpool,5,0,12,12,5,8,56,...,47.2,51.2,16.0,15.4,6.4,4.4,550.6,556.6,15.0,19.0


### 3. Modeling 

dự định là input thông số 2 đội, lịch sử đối đầu và dự đoán thắng thua, số bàn 

In [3]:
X = df[['Team A', 'Team B', 'Goals A Mean', 'Goals B Mean', 'Possession % A Mean', 'Possession % B Mean', 'Shots A Mean', 'Shots B Mean', 'Shots on Target A Mean', 'Shots on Target B Mean', 'Passes A Mean', 'Passes B Mean', 'Fouls A Mean', 'Fouls B Mean']]
# Goals might be difficult to predict, so we can try to predict the outcome of the match
y = df.apply(lambda row: 1 if row['Goals A'] > row['Goals B'] else (-1 if row['Goals A'] < row['Goals B'] else 0), axis=1)
y

0     1
1     0
2    -1
3     1
4     0
     ..
95    1
96    1
97    1
98    1
99   -1
Length: 100, dtype: int64